# Import Libraries

In [1]:
%run utilities/global_class

StatementMeta(, 18, -1, Finished, Available)

In [2]:
%run utilities/package

StatementMeta(, 18, -1, Finished, Available)

In [3]:
#staging file format
container_name = "hanifsystem"
account_name = "hanifdatalake"
stg_fileformat = "parquet"
stg_delimiter = ","
stg_withheader = "true"
stg_main_folder = "synapse/workspaces/AdventureWorks2022/humanresources_employeepayhistory"
#curated file format
cur_fileformat = "delta"
cur_delimiter = ","
cur_withheader = "true"
cur_main_folder = "synapse/workspaces/curated/AdventureWorks/humanresources_employeepayhistory"
#mart file format
mart_fileformat = "parquet"
mart_delimiter = ","
mart_withheader = "true"
mart_main_folder = "synapse/workspaces/datamart/AdventureWorks2022/humanresources_employeepayhistory"

StatementMeta(smallnotebook, 18, 12, Finished, Available)

In [4]:
start_date = datetime.strftime(datetime.now(), '%Y%m%d')
ingest_range_day_structured = -2

date_1 = datetime.strptime(start_date, '%Y%m%d')
result_1 = date_1 + timedelta(days = ingest_range_day_structured)
filter_date = result_1.strftime('%Y%m%d')

print(filter_date)

StatementMeta(smallnotebook, 18, 13, Finished, Available)

20230716


In [5]:
set_stg_path = PathConstructor(container_name, account_name, stg_main_folder)
stg_path = set_stg_path.pathconstructor()
print(stg_path)
set_cur_path = PathConstructor(container_name, account_name, cur_main_folder)
cur_path = set_cur_path.pathconstructor()
print(cur_path)

StatementMeta(smallnotebook, 18, 14, Finished, Available)

abfss://hanifsystem@hanifdatalake.dfs.core.windows.net/synapse/workspaces/AdventureWorks2022/humanresources_employeepayhistory
abfss://hanifsystem@hanifdatalake.dfs.core.windows.net/synapse/workspaces/curated/AdventureWorks/humanresources_employeepayhistory


# Staging Section

In [6]:
#df_stg_without_schema
df_stg_without_schema = spark.read.format(stg_fileformat)\
.option('header',stg_withheader)\
.option('inferSchema','true')\
.load(stg_path)
display(df_stg_without_schema.limit(10))

StatementMeta(smallnotebook, 18, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5fea249f-fba0-4495-9f2f-dca8d6fdd73b)

In [7]:
df_stg_without_schema.printSchema()

StatementMeta(smallnotebook, 18, 16, Finished, Available)

root
 |-- BusinessEntityID: integer (nullable = true)
 |-- RateChangeDate: timestamp (nullable = true)
 |-- Rate: decimal(19,4) (nullable = true)
 |-- PayFrequency: integer (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)



In [8]:
col_name = df_stg_without_schema.columns
lower_col_name = [name.lower() for name in col_name]
df_stg = df_stg_without_schema.toDF(*lower_col_name)

StatementMeta(smallnotebook, 18, 17, Finished, Available)

In [9]:
df_stg.printSchema()

StatementMeta(smallnotebook, 18, 18, Finished, Available)

root
 |-- businessentityid: integer (nullable = true)
 |-- ratechangedate: timestamp (nullable = true)
 |-- rate: decimal(19,4) (nullable = true)
 |-- payfrequency: integer (nullable = true)
 |-- modifieddate: timestamp (nullable = true)



In [10]:
display(df_stg.limit(10))

StatementMeta(smallnotebook, 18, 19, Finished, Available)

SynapseWidget(Synapse.DataFrame, d3c706c2-6d50-4ca2-bc9e-d4b91e39b163)

In [11]:
df_stg =df_stg.withColumnRenamed('modifieddate','modifiedutcdate')
df_stg.printSchema()

StatementMeta(smallnotebook, 18, 20, Finished, Available)

root
 |-- businessentityid: integer (nullable = true)
 |-- ratechangedate: timestamp (nullable = true)
 |-- rate: decimal(19,4) (nullable = true)
 |-- payfrequency: integer (nullable = true)
 |-- modifiedutcdate: timestamp (nullable = true)



In [12]:
partition_column = ["businessentityid"]

df_stg_final = df_stg.withColumn("rank", row_number() \
                                        .over(Window.partitionBy(*partition_column) \
                                        .orderBy(desc("modifiedutcdate")))) \
                     .withColumn("curated_date", F.lit(datetime.now())) \
                     .where("rank == 1").drop("rank")

StatementMeta(smallnotebook, 18, 21, Finished, Available)

# Curated Section

In [13]:
try:
    set_df_cur = ReadFile(cur_path, cur_fileformat, cur_delimiter, cur_withheader)
    df_cur = set_df_cur.readfrompath()
except Exception as ex:
    df_final = df_stg_final.coalesce(1)
    df_final.write.format('delta') \
            .mode('overwrite') \
            .save(cur_path)

    set_df_cur = ReadFile(cur_path, cur_fileformat, cur_delimiter, cur_withheader)
    df_cur = set_df_cur.readfrompath()

StatementMeta(smallnotebook, 18, 22, Finished, Available)

In [14]:
display(df_cur)

StatementMeta(smallnotebook, 18, 23, Finished, Available)

SynapseWidget(Synapse.DataFrame, 990730f5-8ca2-419e-87cb-908c1791ce8b)

In [15]:
df_cur.createOrReplaceTempView("targetTableName")
df_stg_final.createOrReplaceTempView("updatesTableName")

StatementMeta(smallnotebook, 18, 24, Finished, Available)

In [16]:
df_cur.printSchema()

StatementMeta(smallnotebook, 18, 25, Finished, Available)

root
 |-- businessentityid: integer (nullable = true)
 |-- ratechangedate: timestamp (nullable = true)
 |-- rate: decimal(19,4) (nullable = true)
 |-- payfrequency: integer (nullable = true)
 |-- modifiedutcdate: timestamp (nullable = true)
 |-- curated_date: timestamp (nullable = true)



In [17]:
spark.sql("""
        MERGE INTO targetTableName
        USING updatesTableName
        ON date_format(updatesTableName.modifiedutcdate, 'yyyy-MM-dd') >= TO_DATE('{0}','yyyyMMdd') AND
           targetTableName.businessentityid = updatesTableName.businessentityid
        WHEN MATCHED THEN UPDATE SET
                targetTableName.ratechangedate = updatesTableName.ratechangedate,
                targetTableName.rate = updatesTableName.rate,
                targetTableName.payfrequency = updatesTableName.payfrequency,
                targetTableName.modifiedutcdate = updatesTableName.modifiedutcdate,
                targetTableName.curated_date = updatesTableName.curated_date                                
        WHEN NOT MATCHED AND (date_format(updatesTableName.modifiedutcdate, 'yyyy-MM-dd') >= TO_DATE('{1}','yyyyMMdd')) THEN INSERT * """.format(filter_date, filter_date)
)

StatementMeta(smallnotebook, 18, 26, Finished, Available)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

# Mart Section

In [18]:
def WriteFile(df, final_path):
    try: 
        spark.conf.set("mapreduce.fileoutputcommitter.marksuccessfuljobs","false")
        df_final = df.coalesce(1)
        df_final.write.format('parquet') \
                .mode('overwrite') \
                .save(final_path)
                                        
        print("Write Success")
    except Exception as ex:
        print("Write Failed", str(ex))

StatementMeta(smallnotebook, 18, 27, Finished, Available)

In [19]:
set_mart_path = PathConstructor(container_name, account_name, mart_main_folder)
mart_path = set_mart_path.pathconstructor()

print("Source: ", cur_path, "Mart: ", mart_path)

StatementMeta(smallnotebook, 18, 28, Finished, Available)

Source:  abfss://hanifsystem@hanifdatalake.dfs.core.windows.net/synapse/workspaces/curated/AdventureWorks/humanresources_employeepayhistory Mart:  abfss://hanifsystem@hanifdatalake.dfs.core.windows.net/synapse/workspaces/datamart/AdventureWorks2022/humanresources_employeepayhistory


In [20]:
#Create empty dataframe
df_mart = spark.createDataFrame([], StructType([]))
df_mart = df_cur

# print(df_mart.count())

StatementMeta(smallnotebook, 18, 29, Finished, Available)

In [21]:
from dateutil.relativedelta import relativedelta

periode = datetime.now() + relativedelta(months = -2)
periode = periode.strftime("%Y-%m") + "-01"
print(periode)
df_mart_final = df_mart.filter(col("curated_date") >= (lit(periode)))

StatementMeta(smallnotebook, 18, 30, Finished, Available)

2023-05-01


In [22]:
partition_list = df_mart_final.select(year(col("curated_date")).alias("year"), month(col("curated_date")) \
.alias("month")).dropDuplicates().orderBy(col("year").asc()).orderBy(col("month").asc()).collect()

print(partition_list)

StatementMeta(smallnotebook, 18, 31, Finished, Available)

[Row(year=2023, month=7)]


In [23]:
for partition in partition_list:
    final_path = mart_path + '/' + str(partition.year) + str(partition.month).zfill(2)
    print('Partition path', final_path)

    try:
        mssparkutils.fs.rm(final_path, True)
    except Exception as e:
        pass

    df_final = df_mart_final.filter(year(col("curated_date")) == partition.year).filter(month(col("curated_date")) == partition.month)

    WriteFile(df_final, final_path)

StatementMeta(smallnotebook, 18, 32, Finished, Available)

Partition path abfss://hanifsystem@hanifdatalake.dfs.core.windows.net/synapse/workspaces/datamart/AdventureWorks2022/humanresources_employeepayhistory/202307
Write Success


In [24]:
display(df_final)

StatementMeta(smallnotebook, 18, 33, Finished, Available)

SynapseWidget(Synapse.DataFrame, 465f570b-c3e4-49d0-a4de-ecf265c27e80)

In [25]:
df_final.printSchema()

StatementMeta(smallnotebook, 18, 34, Finished, Available)

root
 |-- businessentityid: integer (nullable = true)
 |-- ratechangedate: timestamp (nullable = true)
 |-- rate: decimal(19,4) (nullable = true)
 |-- payfrequency: integer (nullable = true)
 |-- modifiedutcdate: timestamp (nullable = true)
 |-- curated_date: timestamp (nullable = true)

